In [1]:
import os, sqlite3, xlrd
import pandas as pd
from utils_query import *

In [2]:
xls_file = pd.ExcelFile('query.xlsx')
table = xls_file.parse('All Samples')
chrom = [x for x in table['Chr'].unique()[:-1]]
conn = sqlite3.connect('dbase_Sqlite')
cleardb('dbase_Sqlite', 'dataFrame')
table.to_sql('dataFrame', conn)

/Users/leehongkai/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1345: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [11]:
df = pd.read_sql_query("SELECT * FROM Indian limit 5" , conn)
table.columns

Index(['Sample', 'Gene', 'Variant', 'Chr', 'Coordinate', 'Variant Length',
       'Type', 'Genotype', 'Exonic', 'Filters', 'Quality', 'GQX',
       'Alternate Alleles', 'Inherited From', 'Alt Variant Freq', 'Read Depth',
       'Alt Read Depth', 'Allelic Depths', 'Custom Annotation',
       'Custom Annotation 2', 'Custom Annotation 3', 'Custom Annotation 4',
       'Custom Gene Annotation', 'Custom Gene Annotation 2', 'Classification',
       'Google Scholar', 'PubMed', 'UCSC Browser', 'Num Transcripts',
       'Transcript', 'Consequence', 'cDNA Position', 'CDS Position',
       'Protein Position', 'Amino Acids', 'Codons', 'Exon', 'Intron',
       'Transcript HGNC', 'Distance', 'Canonical', 'Sift', 'PolyPhen', 'ENSP',
       'HGVSc', 'HGVSp', 'dbSNP ID', 'Ancestral Allele', 'Allele Freq',
       'Allele Freq Global Minor', 'Global Minor Allele', 'Allele Freq Amr',
       'Allele Freq Asn', 'Allele Freq Af', 'Allele Freq Eur',
       'Allele Freq Evs', 'EVS Coverage', 'EVS Samples', 'Co

In [9]:
query = """
        SELECT 
            dataFrame.*, Indian.ALLELE_FREQ_1, Malay.ALLELE_FREQ_2
        FROM
            dataFrame INNER JOIN Indian ON
            dataFrame.Coordinate=Indian.POS INNER JOIN
            Malay ON dataFrame.Coordinate=Malay.POS
        WHERE
            dataFrame.Chr=1
        """
df = pd.read_sql_query(query , conn)
df

,index,Sample,Gene,Variant,Chr,Coordinate,Variant Length,Type,Genotype,Exonic,...,ClinVar Accession,ClinVar MedGen,ClinVar OMIM,ClinVar Orphanet,ClinVar GeneReviews,ClinVar SnoMedCt ID,Regulatory Feature,HGNC,ALLELE_FREQ_1,ALLELE_FREQ_2
0,1,T1-MS,SDHB,G>T/T,1,17380497,1,snv,hom,yes,...,None,None,None,None,None,None,ENSR00000531518,SDHB,G:0.0277778,T:1
1,1,T1-MS,SDHB,G>T/T,1,17380497,1,snv,hom,yes,...,None,None,None,None,None,None,ENSR00000531518,SDHB,G:0.0277778,T:1
2,2,T1-MS,MUTYH,C>C/G,1,45797505,1,snv,het,yes,...,None,None,None,None,None,None,None,"MUTYH, HPDL",C:0.819444,G:0.416667
3,2,T1-MS,MUTYH,C>C/G,1,45797505,1,snv,het,yes,...,None,None,None,None,None,None,None,"MUTYH, HPDL",C:0.819444,G:0.416667
4,3,T1-MS,MUTYH,C>C/G,1,45797505,1,snv,het,yes,...,None,None,None,None,None,None,None,"MUTYH, HPDL",C:0.819444,G:0.416667
5,3,T1-MS,MUTYH,C>C/G,1,45797505,1,snv,het,yes,...,None,None,None,None,None,None,None,"MUTYH, HPDL",C:0.819444,G:0.416667
6,5,T1-MS,NR5A2,A>A/G,1,200007432,1,snv,het,no,...,None,None,None,None,None,None,None,NR5A2,A:0.541667,G:0.526042
7,5,T1-MS,NR5A2,A>A/G,1,200007432,1,snv,het,no,...,None,None,None,None,None,None,None,NR5A2,A:0.541667,G:0.526042
8,326,T2-HHE,PRDM2,T>T/C,1,14096821,1,snv,het,no,...,None,None,None,None,None,None,None,PRDM2,T:0.791667,C:0.197917
9,326,T2-HHE,PRDM2,T>T/C,1,14096821,1,snv,het,no,...,None,None,None,None,None,None,None,PRDM2,T:0.791667,C:0.197917
